In [2]:
import os
import numpy as np
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "2" 

In [315]:
file1 = open('../../datasets/RCV1/llm_cluster_result/update_labelspace.txt', 'r')
documents = file1.readlines()  

In [316]:
documents[:4]

['Airline Industry\n', 'Politics\n', 'Sports\n', 'Intelligence Agencies\n']

In [317]:
file2 = open('../../datasets/RCV1/train_label.txt', 'r')
labels = file2.readlines() 

In [318]:
predict_label_space = []
for doc in documents:
    predict_label_space.append(doc.strip())

In [208]:
# predict_label_space = []
# for doc in documents:
#     label_list = doc.strip().split(': ')[1]
#     if not label_list in predict_label_space:
#         predict_label_space.append(label_list)

In [319]:
file2 = open('../../datasets/RCV1-V2/rcv1.topics.hier.orig.txt', 'r')
labels = file2.readlines() 
# file2 = open('../../datasets/AAPD/true_labels.txt', 'r')
# labels = file2.readlines()   

In [320]:
true_label = []
for row in labels[1:]:
    new_label = row.strip().split('child-description: ')[1].lower()
    true_label.append(new_label)

In [321]:
true_label[:5]

['strategy/plans',
 'legal/judicial',
 'regulation/policy',
 'share listings',
 'performance']

In [282]:
label_dic = {}
for row in labels:
    key = row.strip().split(';')[0].split('.')[0]
    #key + '.' + 
    label_dic[row.strip().split(';')[0]] = row.strip().split(';')[1][1:]
true_label = []
for key in label_dic:
    if not label_dic[key] in true_label:
        true_label.append(label_dic[key])
true_label[:5]

['Information Retrieval',
 'Methodology',
 'Quantum Physics',
 'Information Theory',
 'Applications']

In [5]:
true_label = []
count = 0
for row in labels[2000:3000]:
    label_list = row.strip().split(' ')
    if len(label_list) == 1:
        count +=1
    for label in label_list:
        if not label in true_label:
            true_label.append(label)
true_label[:5]

['money-fx', 'earn', 'acq', 'sugar', 'trade']

In [6]:
count

852

In [295]:
true_label = []
for row in labels:
    label_list = row.strip().split(' ')
    for label in label_list:
        if not label in true_label:
            true_label.append(label)
true_label[:5]

# true_label = []
# for row in labels:
#     true_label.append(row.strip().split('\t')[1])
# true_label[:5]

['cocoa', 'sorghum', 'oat', 'barley', 'corn']

In [229]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

query_embedding = model.encode('teethers')
passage_embedding = model.encode(['baby_dental_care'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Similarity: tensor([[0.4893]])


In [322]:
passage_embedding = model.encode(predict_label_space)

In [323]:
sim_matrix = np.empty((0,len(predict_label_space)))
for i in range(len(true_label)):
    query_embedding = model.encode(true_label[i])
    sim_matrix = np.append(sim_matrix, util.dot_score(query_embedding, passage_embedding).numpy(), 0)

In [324]:
sim_matrix.shape

(103, 110)

In [325]:
sim_list = []
for i, sim_score in enumerate(sim_matrix):
    rank_list = np.argsort(sim_score)[-40:]
    cur_list = [true_label[i]]
    for index in rank_list:
        cur_list.append((sim_score[index], predict_label_space[index]))
    if cur_list[-1][0] >= 0.6:
        sim_list.append(cur_list)
sim_list[:2]

[['legal/judicial',
  (0.2560056447982788, 'Technology/Internet'),
  (0.25780072808265686, 'Operations'),
  (0.25806641578674316, 'Protests/Strikes'),
  (0.2604048550128937, 'Historical Events'),
  (0.2673090100288391, 'Resignation'),
  (0.2749345302581787, 'Terrorism'),
  (0.27502572536468506, 'Sales/Revenue'),
  (0.27564650774002075, 'Labor/Workforce'),
  (0.27792787551879883, 'Auctions'),
  (0.27962490916252136, 'Privatization'),
  (0.2804669737815857, 'Geopolitics'),
  (0.28190767765045166, 'Death'),
  (0.2854553461074829, 'Environmental'),
  (0.28918778896331787, 'European Union'),
  (0.29928943514823914, 'Financial/Economic'),
  (0.3115212321281433, 'Sports'),
  (0.3132980465888977, 'Trade Regulations'),
  (0.3178219795227051, 'Competition'),
  (0.31876689195632935, 'Location'),
  (0.3215720057487488, 'Education/Training'),
  (0.3246457278728485, 'Intelligence Agencies'),
  (0.32509419322013855, 'Military'),
  (0.3338983654975891, 'Real Estate'),
  (0.34357425570487976, 'Literatu

In [326]:
pair_list = []
for line in sim_list:
    true_label = line[0]
    for row in line[1:]:
        if row[0] >= 0.6:
            pair_list.append([(true_label, row[1]), row[0]])
sorted_pair_list = sorted(pair_list, key=lambda x: x[1], reverse = True)
sorted_pair_list[:5]

[[('sports', 'Sports'), 1.0000001192092896],
 [('mergers/acquisitions', 'Mergers/Acquisitions'), 1.0],
 [('international relations', 'International Relations'), 1.0],
 [('fashion', 'Fashion'), 1.0],
 [('health', 'Health'), 1.0]]

In [327]:
refine_pair_list = []
while len(sorted_pair_list) > 0:
    refine_pair_list.append([sorted_pair_list[0][0], sorted_pair_list[0][1]])
    ground_truth = sorted_pair_list[0][0][0]
    prediction = sorted_pair_list[0][0][1]
    sorted_pair_list.pop(0)
    delete_list = []
    for row in sorted_pair_list:
        if row[0][0] == ground_truth or row[0][1] == prediction:
            delete_list.append(row)
    for item in delete_list:
        sorted_pair_list.remove(item)
refine_pair_list[:5]

[[('sports', 'Sports'), 1.0000001192092896],
 [('mergers/acquisitions', 'Mergers/Acquisitions'), 1.0],
 [('international relations', 'International Relations'), 1.0],
 [('fashion', 'Fashion'), 1.0],
 [('health', 'Health'), 1.0]]

In [328]:
len(refine_pair_list)

44

In [1]:
44/103

0.42718446601941745